In [100]:
import pandas as pd
import operator
import time
import random
import numpy as np
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
random.seed(0)


df = pd.read_csv('/Users/dashabalashova/1_1_5_ML_Bioinf/data/cancer.csv')
Y = df['label']
df = df.drop('label', axis=1)
col_list=list(df.columns)
def split_into_clusters(df, centers, K):
    #print(centers)
    d = df
    for k in range(0,K):
        d1 = (df - np.array(centers.iloc[k])).pow(2).sum(1).pow(0.5)
        d = pd.concat([d, d1], axis=1)
    d.columns = col_list + list(range(0, K))
    e = d[list(range(0, K))].idxmin(axis=1)
    d = pd.concat((d, e.rename('cluster')), axis=1)[col_list+['cluster']]
    new_centers = d.groupby(['cluster']).mean()
    a = sum((centers-new_centers).pow(2).sum(1).pow(0.5))
    return d, new_centers, a



def main(K):
    a=1
    centers = []
    for n in random.sample(range(1, df.shape[0]), K):
        centers.append(list(df.iloc[n]))    
    centers = pd.DataFrame(centers, columns=col_list)
    while a>0.01:
        d, centers, a = split_into_clusters(df, centers, K)
    d['cluster'] = pd.to_numeric(d['cluster'])
    return d

def dist(n, d):
    df=d[d['cluster']==n].reset_index(drop=True)
    df = df.drop('cluster', axis=1)
    df = df.rename(str.lower, axis='columns')
    l = df.shape[0]
    T = KDTree(df)
    di = []
    if l==1:
        return 0, list(df.mean()[col_list])
    else:
        for t in df.index:
            v1=df.loc[t]
            di.append(T.query(v1, k=l)[0][-1])
        return max(di), list(df.mean()[col_list])

def Dunn_index(d):
    st_1=[]
    st_2=[]
    st_3 = []
    for n in d.groupby('cluster').count().index:
        if n>=0:
            a=dist(n, d)
            st_1.append(a[0])
            st_2.append(np.array(a[1]))
    for n in range(0,len(st_1)):
        for m in range(n+1,len(st_1)):
            st_3.append(np.linalg.norm(st_2[n]-st_2[m]))
    return min(st_3)/max(st_1)


def Purity(d, Y):
    XY = pd.concat([d, Y], axis=1)
    XY2 = pd.pivot_table(XY[['cluster', 'label']], index=['cluster'], columns=['label'], aggfunc=len, fill_value=0)
    XY2['max']=XY2.apply(lambda x: max(x['B'],x['M']), axis=1)
    return XY2.sum(axis = 0)['max']/(XY2.sum(axis = 0)['B']+XY2.sum(axis = 0)['M'])
    
    
    

for n in range(2,11):
    d = main(n) 
    print('number of clusters =', n, ', purity =', Purity(d, Y))
    print()


number of clusters = 2 , purity = 0.8541300527240774

number of clusters = 3 , purity = 0.8875219683655536

number of clusters = 4 , purity = 0.8347978910369068

number of clusters = 5 , purity = 0.8910369068541301

number of clusters = 6 , purity = 0.8769771528998243

number of clusters = 7 , purity = 0.8910369068541301

number of clusters = 8 , purity = 0.8910369068541301

number of clusters = 9 , purity = 0.9086115992970123

number of clusters = 10 , purity = 0.9103690685413005



In [102]:
from sklearn import preprocessing

df = pd.read_csv('/Users/dashabalashova/1_1_5_ML_Bioinf/data/cancer.csv')
Y = df['label']
df = df.drop('label', axis=1)

df = pd.DataFrame(preprocessing.normalize(df))

for n in range(2,11):
    d = main(n) 
    print('number of clusters =', n, ', purity =', Purity(d, Y))
    print()

number of clusters = 2 , purity = 0.8769771528998243

number of clusters = 3 , purity = 0.8699472759226714

number of clusters = 4 , purity = 0.8066783831282952

number of clusters = 5 , purity = 0.8998242530755711

number of clusters = 6 , purity = 0.8945518453427065

number of clusters = 7 , purity = 0.8998242530755711

number of clusters = 8 , purity = 0.8699472759226714

number of clusters = 9 , purity = 0.9068541300527241

number of clusters = 10 , purity = 0.8980667838312829

